In [1]:
%load_ext autoreload
%autoreload 2

import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision

# To view tensorboard metrics
# tensorboard --logdir=log --port=6006 --bind_all
from torch.utils.tensorboard import SummaryWriter
from functools import partial
from evolver import CrossoverType, MutationType, MatrixEvolver
from models.unet import UNet
from metrics import accuracy, iou
from dataset_utils import PartitionType
from cuda_utils import maybe_get_cuda_device, clear_gpu
from landcover_dataloader import get_landcover_dataloaders

In [ ]:
# Define directories for data, logging and model saving.
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, "data/landcover_small")
log_dir = os.path.join(os.getcwd(), "/log")
model_dir = os.path.join(os.getcwd(), "/saved_models")

In [3]:
# Create DataLoaders for each partition of Landcover data.
dataloader_params = {
    'batch_size': 16,
    'shuffle': True,
    'num_workers': 4,
    'pin_memory': True}

partition_types = [PartitionType.TRAIN, PartitionType.VALIDATION, 
                   PartitionType.FINETUNING, PartitionType.TEST]
data_loaders = get_landcover_dataloaders(dataset_dir, 
                                         partition_types,
                                         dataloader_params)
train_loader = data_loaders[0]
validation_loader = data_loaders[1]
finetuning_loader = data_loaders[2]
test_loader = data_loaders[3]

In [9]:
def train(engine, batch):
    """"""
    with torch.enable_grad():
        batch_x, batch_y = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
            
def evaluate(engine, batch):
    """"""
    model.eval()
    with torch.no_grad():
        batch_x, batch_y = batch[0].to(device), batch[1].to(device)
        outputs = model(batch_x)
        return outputs, batch_y

In [10]:
# Get GPU device if available.
device = cuda_utils.maybe_get_cuda_device()

params = {
    'max_epochs': 1,
    'n_classes': 4,
    'in_channels': 4,
    'depth': 4,
    'learning_rate': 0.01,
    'momentum': 0.8,
    'log_steps': 5
}

metrics = {
    "accuracy": accuracy_fn
}

cuda_utils.clear_gpu()    
model = UNet(in_channels = params['in_channels'],
             n_classes = params['n_classes'],
             depth = params['depth'])
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=params['learning_rate'],
                            momentum=params['momentum'])

In [ ]:
# Train Model
for epoch in range(params['max_epochs']):
    train_loss, train_reports = train(model, optimizer, loss_fn,
                                      train_loader, metrics, params['log_steps'], device)
    validate_loss, validate_reports = evaluate(model, loss_fn, validation_loader, metrics, device)

In [11]:
# Fine tune with back prop
# TODO move to finetuning directory
for epoch in range(params['max_epochs']):
    finetuning_loss, finetuning_reports = train(model, optimizer, loss_fn,
                                                finetuning_loader, metrics, params['log_steps'], device)
    
    test_loss, test_reports = evaluate(model, loss_fn, test_loader, metrics, device)
    
# Finetine with Dropout
dropout_masks = {
    'start': [256, 256],
#     'down_0': None,
#     'down_1': None,
#     'down_2': None,
#     'down_3': None,
#     'down_4': None,
#     'up_0': None,
#     'up_1': None,
#     'up_2': None,
#     'up_3': None,
#     'end': None,   
}

finetuning_params = {
    "n_generations": 100
    "n_children": 10
}



evolver = MatrixEvolver([m for k, m in dropout_masks.items() if m is not None],
                        CrossoverType.UNIFORM, MutationType.FLIP_BIT)

def dropout_finetune(engine, batch):
    model.eval()
    with torch.no_grad():
        batch_x, batch_y = batch[0].to(device), batch[1].to(device)
        for child in range(params['n_children']):
            child_masks = evolver.spawn_child()
            model.set_dropout_masks({k: torch.tensor(child_masks[i], 
                                                     device=device,
                                                     dtype=torch.float) for i, k in enumerate(dropout_masks.keys())})

            outputs = model(batch_x)
            loss = loss_fn(outputs, batch_y)
            total_loss += loss.item()
            evolver.add_child(child_masks, total_loss)

        evolver.update_parents()

    return pass


for epoch in range(finetuning_params['max_epochs']):
    dropout_finetune(model, loss_fn, data_loader, metrics, dropout_masks, params, device)
    test_loss, test_reports = evaluate(model, loss_fn, test_loader, metrics, device)

Train:  1.4302207231521606
Train:  1.1587666273117065
Train:  1.4722106456756592
Train:  1.4438084363937378
Train:  1.1136218309402466
Train:  1.2759853601455688
Train:  0.9879368543624878
Train:  1.1005209684371948
